In [ ]:
import os
os.chdir('')

In [ ]:
!pip install transformers==4.32.0 accelerate==0.25.0 tiktoken einops transformers_stream_generator==0.0.4 scipy modelscope optimum==1.12.0 peft==0.6.2 deepspeed==0.12.4 cohere py-cpuinfo psutil==5.9.6 torch==2.1.1 pandas==2.1.3
!pip install mpi4py
!pip install flash-attn --no-build-isolation

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM
import psutil
import torch
import shutil
from modelscope import snapshot_download
import subprocess
import json
import pickle

In [ ]:
def get_model(model_id):
    """从modelscope拉取模型，并将模型移动到model文件夹内"""

    model_dir = snapshot_download(model_id)
    shutil.move(model_dir, 'model')
    model_path = os.path.abspath(os.path.join('model', os.path.basename(model_dir)))
    return model_path

In [ ]:
def get_test(test_data, model, tokenizer):
    # 使用测试集数据进行推理
    def get_response(row):
        response, _ = model.chat(tokenizer, row['question'], history=None)
        return response
    test_data['result_new'] = test_data.apply(get_response, axis=1)
    # 重置索引并将字符串的json结果分割成为reason, sentiment, impact三列
    test_data['result_new'] = test_data['result_new'].map(lambda x: json.loads(x.replace("'", '"')))
    test_data = test_data.reset_index(drop=True)
    test_all = pd.concat([test_data, pd.json_normalize(test_data['result_new'])], axis=1)
    # 将新的列后面加_new后缀
    rename_dict = {col:col+'_new' for col in ['reason', 'sentiment', 'impact']}
    test_all = test_all.rename(columns=rename_dict)
    # 进行数据格式转换
    test_all[['sentiment_new', 'impact_new']] = test_all[['sentiment_new', 'impact_new']].apply(pd.to_numeric)
    # 计算情绪分数的误差
    def get_sentiment_error(row):
        old = row['sentiment_old']
        new = row['sentiment_new']
        if old * new >= 0:
            error = new - old
        else:
            error = 1 if abs(new - old) <= 1 else abs(new - old)
        return error ** 2
    # 计算影响力分数的误差
    def get_impact_error(row):
        old = row['impact_old']
        new = row['impact_new']
        return abs(new - old) ** 2
    test_all['sentiment_se'] = test_all.apply(get_sentiment_error, axis=1)
    test_all['sentiment_se_0.1'] = test_all['sentiment_se'] > 0.1
    test_all['impact_se'] = test_all.apply(get_impact_error, axis=1)
    test_all['impact_se_0.1'] = test_all['impact_se'] > 0.1
    return test_all

In [ ]:
# 训练文件路径的列表
train_path = 'data/train'
train_list = [os.path.join(train_path, file) for file in os.listdir(train_path)]
# 测试文件的路径
test_path = 'data/test'
test_list = [pd.read_pickle(os.path.join(test_path, file)) for file in os.listdir(test_path)]

In [ ]:
model_path = get_model('qwen/Qwen-1_8B-Chat')
# model_path = ''
adapter_path = f"adapter/single"
model_path

In [ ]:
!bash finetune_lora_single_gpu.sh -m  -d data/train/train_00.json -o adapter/single

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
adapter_path,
device_map="auto",
trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
test_list = [get_test(data, model, tokenizer) for data in test_list]
with open('outcome.pkl', 'wb') as f:
    pickle.dump(test_list, f)